# Executing commands and Playbooks at Nodes and using Role System

This notebook....

In [1]:
import sys
sys.path.append('..')
!pip list

Package                       Version
----------------------------- -----------
alabaster                     0.7.12
ansible                       3.4.0
ansible-base                  2.10.9
ansible-runner                1.4.7
anyio                         3.1.0
argcomplete                   1.12.3
argon2-cffi                   20.1.0
astroid                       2.5.6
async-generator               1.10
attrs                         21.2.0
Babel                         2.9.1
backcall                      0.2.0
bcrypt                        3.2.0
bleach                        3.3.0
boto                          2.49.0
boto3                         1.17.73
botocore                      1.20.73
certifi                       2020.12.5
cffi                          1.14.5
chardet                       4.0.0
click                         8.0.0
coloredlogs                   15.0
contextlib2                   0.6.0.post1
cryptography                  3.4.7
dacite                        1.6.0
d

In [7]:
import yaml
from dataclasses import asdict
from app.cli.modules.node import get_config_db, get_node_manager
from app.cli.modules.role import get_role_manager
from clap.utils import float_time_to_string
from clap.executor import SSHCommandExecutor, AnsiblePlaybookExecutor

In [3]:
configuration_db = get_config_db()
node_manager = get_node_manager()

In [12]:
nodes = node_manager.get_all_nodes()
for node in nodes:
    print(f"Node ID: {node.node_id} ({node.nickname}); status: {node.status}; "
          f"IP: {node.ip}; type: {node.configuration.instance.instance_config_id}")

Node ID: 8133c6f7f9ca48258d1e0f01e11326f6 (GeorgeHerndon); status: reachable; IP: 34.229.255.112; type: type-a
Node ID: 07f4a369663f48d48254f2ad4c5abbfe (HenryRoy); status: reachable; IP: 54.144.141.190; type: type-a


In [22]:
command_to_execute = """
git clone https://github.com/lmcad-unicamp/CLAP.git
cd CLAP
chmod +x install.sh
echo Clonned CLAP into `pwd`
echo Done
"""
private_path = node_manager.private_path

executor = SSHCommandExecutor(command_to_execute, nodes, private_path)
result = executor.run()

print(yaml.dump(result, indent=4, sort_keys=True))

07f4a369663f48d48254f2ad4c5abbfe:
    error: null
    ok: true
    return_code: 0
    stderr_lines:
    - 'Cloning into ''CLAP''...

        '
    stdout_lines:
    - 'Clonned CLAP into /home/ubuntu/CLAP

        '
    - 'Done

        '
8133c6f7f9ca48258d1e0f01e11326f6:
    error: null
    ok: true
    return_code: 0
    stderr_lines:
    - 'Cloning into ''CLAP''...

        '
    stdout_lines:
    - 'Clonned CLAP into /home/ubuntu/CLAP

        '
    - 'Done

        '



07f4a369663f48d48254f2ad4c5abbfe:
    error: null
    ok: true
    return_code: 0
    stderr_lines:
    - 'Cloning into ''CLAP''...

        '
    stdout_lines:
    - 'Clonned CLAP into /home/ubuntu/CLAP

        '
8133c6f7f9ca48258d1e0f01e11326f6:
    error: null
    ok: true
    return_code: 0
    stderr_lines:
    - 'Cloning into ''CLAP''...

        '
    stdout_lines:
    - 'Clonned CLAP into /home/ubuntu/CLAP

        '



In [ ]:
role_manager = get_role_manager()

In [4]:
!cat ~/.clap/roles/actions.d/commands-common.yaml

---
actions:
  install-packages:
    playbook: roles/commands-common_install.yml
    description: Install packages in nodes
    vars:
    - name: packages
      description: Packages to install (comma separated)

  copy:
    playbook: roles/commands-common_copy.yml
    description: Copy files from localhost to remote hosts
    vars:
    - name: src
      description: Source files/directory to be copied
    - name: dest
      description: Destination directory where files will be placed

  fetch:
    playbook: roles/commands-common_fetch.yml
    description: Fetch files from remote hosts to localhosts
    vars:
    - name: src
      description: Source files/directory to be fetched
    - name: dest
      description: Destination directory where files will be placed

  reboot:
    playbook: roles/commands-common_reboot.yml
    description: Reboot a machine

  run-command:
    playbook: roles/commands-common_run-command.yml
    description: Run a shell command in remote hosts
    vars:
  

In [6]:

role_manager.roles

{'commands-common': Role(actions={'install-packages': RoleActionInfo(playbook='roles/commands-common_install.yml', description='Install packages in nodes', vars=[RoleVariableInfo(name='packages', description='Packages to install (comma separated)', optional=None)]), 'copy': RoleActionInfo(playbook='roles/commands-common_copy.yml', description='Copy files from localhost to remote hosts', vars=[RoleVariableInfo(name='src', description='Source files/directory to be copied', optional=None), RoleVariableInfo(name='dest', description='Destination directory where files will be placed', optional=None)]), 'fetch': RoleActionInfo(playbook='roles/commands-common_fetch.yml', description='Fetch files from remote hosts to localhosts', vars=[RoleVariableInfo(name='src', description='Source files/directory to be fetched', optional=None), RoleVariableInfo(name='dest', description='Destination directory where files will be placed', optional=None)]), 'reboot': RoleActionInfo(playbook='roles/commands-comm